In [313]:
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
import MeCab
import nltk
import pickle
from pandas import Series

#윈도우 mecab 불러오기

from konlpy.tag import Okt
import re

#형태소 분석을 위한 객체 생성
kkma=Kkma()
komoran=Komoran()
hannanum=Hannanum()
#윈도우에서 Mecab생성
mecab=MeCab.Tagger()
okt=Okt()
stop_words=[
    '이','있','하','것','들','그','되','이','보','않','없','나','같','없','가','때','들','주','등','같',
    '년','한','지','오','말','일','나오','그렇','위하','때문','그것','두','알','그러나','받','못하','요즘',
    '그런','또','더','그리고','좋','중','따르','가지','씨','시키','살','될','안','데','내','경우','다시','그녀',
    '이런','앞','나','된','잘','놓','위','위해','했','받','해','대해','해서','통해','후기','사','점','끝',
    '이번','큰','와서','동안','지금','오늘','코로나','네이버','위한','넣','크','맞','날','할','속','남','그때',
    '한다','작','적','많','왔','해도','된다','전','이르','후'
]
print(len(stop_words))


100


In [58]:
#여가 딕셔너리 검색어 생성(검색/출력용)
big_cat=[
    '문화예술관람활동','문화예술참여활동','스포츠관람활동',
    '스포츠참여활동','관광활동','취미/오락활동',
    '휴식활동','사회/기타활동'
]
leisure_dict_search=[
    #문화예술관람활동
    ['미술_전시회_관람','사진_전시회_관람','건축_전시회_관람','디자인_전시회_관람',
    '박물관_관람',
    '클래식_음악_연주회_관람','오페라_음악_연주회_관람',
    '국악_전통공연_관람','민속놀이_공연_관람',
    '연극공연_관람','뮤지컬_관람',
    '무용_공연_관람',
    '영화_관람',
    '쇼_공연_관람','콘서트_공연_관람','마술_쇼_공연_관람'],
    #문화예술참여활동
    ['문학_행사_참여',
    '글짓기_참여' ,'독서_토론_참여',
    '그림_활동_참여','서예_활동_참여','조각_활동_참여','디자인_활동_참여','도예_활동_참여','만화_활동_참여',
    '악기_연주_참여','노래교실 참여',
    '춤_참여','무용_참여','발레_참여','한국무용_참여','현대무용_참여','방송댄스_참여','스트릿댄스_참여','비보잉_참여',
    '사진_촬영',
    '사물놀이_참여','줄타기_참여'],
    #스포츠관람활동
    ['축구_경기장_방문_관람' ,'야구_경기장_방문_관람','농구_경기장_방문_관람','배구_경기장_방문_관람',
    '축구_경기_티비_관람','야구_경기_티비_관람','농구_경기_티비_관람','배구_경기_티비_관람',
    '태권도_경기_관람','유도_경기_관람','합기도_경기_관람','검도_경기_관람','권투_경기_관람',
    'e-스포츠_경기_관람'],
    #스포츠참여활동
    ['농구_참가','배구_참가','야구_참가','축구_참가','족구_참여',
    '테니스_참여','스쿼시_참여',
    '당구_참여','포켓볼_참여',
    '볼링_참여','탁구_참여',
    '골프_참여',
    '수영_참여' ,
    '윈드서핑_참여','수상스키_참여','스킨스쿠버다이빙_참여','래프팅_참여','요트_참여',
    '스노보드_참여','스키_참여',
    '아이스스케이트_참여','아이스하키_참여',
    '헬스_참여','보디빌딩_참여','에어로빅_참여',
    '요가_참여','필라테스_참여','태보_운동_참여',
    '배드민턴_참여','줄넘기_참여','맨손_체조_참여','스트레칭_참여','훌라후프_참여',
    '육상_참여','조깅_참여','빠르게_걷기_참여',
    '태권도_참여','유도_경기_참가','합기도_참여','검도_참여','권투_참여',
    '탱고_참여','왈츠_댄스_참여','자이브_참여','맘보_참여','폴카_댄스_참여','차차차_댄스_참여',
    '사이클링_참여','자전거_참여',
    '인라인_스케이트_참여',
    '승마_참여','암벽등반_참여','철인삼종경기_참여','서바이벌_참여'],
    #관광활동
    ['문화유적_방문',
    '자연명승_방문','풍경_관람',
    '삼림욕',
    '국내캠핑',
    '해외여행',
    '소풍','야유회',
    '온천','해수욕',
    '유람선_타기',
    '테마파크_가기','놀이공원_가기','동물원_가기','식물원_가기',
    '지역축제_참가',
    '자동차_드라이브'],
    #취미오락활동
    ['콜렉션','스크랩북',
    '십자수','비즈공예', 'DIY','꽃꽂이',
    '요리하기','다도',
    '반려동물_키우기',
    '노래방_가기',
    '집_인테리어','자동차_인테리어',
    '등산',
    '낚시',
    '홈페이지_관리','블로그_관리',
    '인터넷_검색','1인_미디어_제작','SNS',
    '온라인_게임','모바일_게임','콘솔_게임',
    '보드게임','퍼즐','큐브_맞추기',
    '바둑','장기','체스',
    '경마','경륜','카드놀이','고스톱','마작','복권구입',
    '쇼핑','외식',
    '음주',
    '독서','웹소설',
    '만화보기','웹툰보기',
    '피부관리','헤어관리','네일아트','마사지','성형',
    '어학_공부','기술_공부','자격증_공부',
    '방탈출_카페_체험','VR_카페_체험','낚시카페_체험',
    '원예','화단_가꾸기'],
    #휴식활동
    ['산책','걷기',
    '목욕','사우나','찜질방',
    '낮잠',
    'TV_시청',
    '모바일_컨텐츠','동영상_시청',
    '라디오_청취','팟캐스트_청취',
    '음악_감상',
    '신문_보기','잡지_보기',
    '아무것도_안_하기'],
    #사회 및 기타활동
    ['사회봉사',
    '종교활동',
    '클럽_가기','나이트_가기','디스코_가기','캬바레_가기',
    '가족_방문','친지_방문',
    '잡담','통화하기','문자보내기',
    '계모임','동창회','사교파티',
    '친구_만남','이성교제','미팅','소개팅',
    '동호회_모임']
]

In [3]:
online_keywords=['온라인', '혼자', '집', '통화', '유튜브','툰','소설','읽','문자','만들','관리','안정','신규',
                 '채널', '휴대폰', '블로그', '잠', '티비', '관람','화면','영상','개인','명상','업데이트','보상',
                '웹', '사이트', '간단', '홈', '중계', '시청','비대', '전화','미디어','쉬','간단','캐릭터','콘텐츠',
                 '문자', '발송', '인터넷', '청취','셀프','페이스북','그램','팟캐스트','모바일','손','유저','스킬']
offline_keywords=['오프라인', '공간', '곳', '길', '입장', '방문', '룸','호텔','도착','경주','건축','관람','방','층','타','들어가','레슨',
                 '분위기', '여행', '관광', '지역', '시설', '출입', '모임','사람','느낌','전시','공연','데이트','훈련',
                 '이용', '체육관', '공원', '풍경', '가족', '스포츠', '산','메이트','외식','테마','전시회','주민','예매','참여','판',
                  '수술','캠프','직관','관람객','예약','만나','코치','조','부','무대','걷','경기','대회','춤','사람']
print(len(online_keywords), len(offline_keywords))

50 60


In [7]:
import pandas as pd
weekday=pd.read_csv('C:/Users/mitha/OneDrive/바탕 화면/평일_여가활동_유형.csv',encoding='CP949')
weekend=pd.read_csv('C:/Users/mitha/OneDrive/바탕 화면/휴일_여가활동_유형.csv',encoding='CP949')

In [8]:
weekday

,구분,휴식활동,취미/오락활동,사회/기타활동,관광활동,스포츠참여활동,스포츠관람활동,문화예술관람활동,문화예술참여활동
0,전체,98.9,94.3,93.4,50.5,48.4,43.7,37.7,17.5
1,남성,98.7,94.3,92.3,48.5,53.3,62.2,36.0,15.4
2,여성,99.1,94.2,94.6,52.5,43.5,25.4,39.4,19.6
3,15-19세,99.1,98.2,94.7,36.3,57.1,39.9,45.4,30.2
4,20대,99.0,97.7,94.7,49.3,60.9,50.4,60.6,24.3
5,30대,98.5,95.6,92.1,53.3,53.4,47.7,48.7,20.4
6,40대,98.6,95.4,92.3,54.8,51.8,47.9,42.1,17.1
7,50대,99.1,93.8,94.1,53.2,46.1,45.4,33.6,15.1
8,60대,98.7,92.5,94.1,51.6,40.6,40.2,22.6,13.0
9,70대이상,99.6,87.5,92.9,42.6,30.0,27.0,8.4,8.8


In [33]:
weekday=weekday.set_index('구분')
weekend=weekend.set_index('구분')

KeyError: "None of ['구분'] are in the columns"

In [34]:
weekday

,휴식활동,취미/오락활동,사회/기타활동,관광활동,스포츠참여활동,스포츠관람활동,문화예술관람활동,문화예술참여활동
구분,,,,,,,,
전체,98.9,94.3,93.4,50.5,48.4,43.7,37.7,17.5
남성,98.7,94.3,92.3,48.5,53.3,62.2,36.0,15.4
여성,99.1,94.2,94.6,52.5,43.5,25.4,39.4,19.6
15-19세,99.1,98.2,94.7,36.3,57.1,39.9,45.4,30.2
20대,99.0,97.7,94.7,49.3,60.9,50.4,60.6,24.3
30대,98.5,95.6,92.1,53.3,53.4,47.7,48.7,20.4
40대,98.6,95.4,92.3,54.8,51.8,47.9,42.1,17.1
50대,99.1,93.8,94.1,53.2,46.1,45.4,33.6,15.1
60대,98.7,92.5,94.1,51.6,40.6,40.2,22.6,13.0


In [152]:
#지금껏 구한 상관도 불러오기
fread=open("./Leisures/Description_Data_New/correlation.pkl", 'rb')
colarray=pickle.load(fread)
colarray

array([[1.        , 0.90181135, 0.60888576, ..., 0.10823162, 0.05281323,
        0.16562423],
       [0.90181135, 1.        , 0.6271338 , ..., 0.13405379, 0.078619  ,
        0.17115989],
       [0.60888576, 0.6271338 , 1.        , ..., 0.20524733, 0.06209836,
        0.13319471],
       ...,
       [0.10823162, 0.13405379, 0.20524733, ..., 1.        , 0.34737064,
        0.28993141],
       [0.05281323, 0.078619  , 0.06209836, ..., 0.34737064, 1.        ,
        0.28343072],
       [0.16562423, 0.17115989, 0.13319471, ..., 0.28993141, 0.28343072,
        1.        ]])

- 컬럼 만들어봅시다

In [154]:
columns=[[],[]]
for index,row in enumerate(leisure_dict_search):
    for obj in row:
        columns[0].append(big_cat[index])
        columns[1].append(obj)

In [155]:
correlation=pd.DataFrame(colarray,columns=columns,index=columns)
correlation

문화예술관람활동                                           \
                    미술_전시회_관람 사진_전시회_관람 건축_전시회_관람 디자인_전시회_관람    박물관_관람   
문화예술관람활동 미술_전시회_관람   1.000000  0.901811  0.608886   0.781737  0.422173   
         사진_전시회_관람   0.901811  1.000000  0.627134   0.831878  0.481548   
         건축_전시회_관람   0.608886  0.627134  1.000000   0.682047  0.388383   
         디자인_전시회_관람  0.781737  0.831878  0.682047   1.000000  0.460597   
         박물관_관람      0.422173  0.481548  0.388383   0.460597  1.000000   
...                       ...       ...       ...        ...       ...   
사회/기타활동  친구_만남       0.084373  0.119339  0.074717   0.109248  0.129643   
         이성교제        0.031545  0.034713  0.024502   0.033300  0.032987   
         미팅          0.108232  0.134054  0.205247   0.166119  0.120814   
         소개팅         0.052813  0.078619  0.062098   0.086900  0.066583   
         동호회_모임      0.165624  0.171160  0.133195   0.162514  0.133329   

                                                                       \
                    클래식_음악_연주회_관람 오페라_음악_연주회_관람 국악_전통공연_관람 민속놀이_공연_관람   
문화예술관람활동 미술_전시회_관람       0.153628      0.149917   0.121489   0.220757   
         사진_전시회_관람       0.145338      0.139008   0.113648   0.241190   
         건축_전시회_관람       0.112477      0.108082   0.099555   0.219725   
         디자인_전시회_관람      0.139484      0.133906   0.106327   0.227851   
         박물관_관람          0.105126      0.104950   0.174615   0.354507   
...                           ...           ...        ...        ...   
사회/기타활동  친구_만남           0.051113      0.043145   0.043504   0.073876   
         이성교제            0.028968      0.029836   0.015472   0.022057   
         미팅              0.056009      0.042689   0.054082   0.093369   
         소개팅             0.027096      0.021939   0.020806   0.045822   
         동호회_모임          0.069557      0.069696   0.056026   0.107897   

                               ...   사회/기타활동                                \
                      연극공연_관람  ...      통화하기     문자보내기       계모임       동창회   
문화예술관람활동 미술_전시회_관람   0.088846  ...  0.029996  0.015479  0.039317  0.059336   
         사진_전시회_관람   0.099477  ...  0.037214  0.018288  0.056720  0.070297   
         건축_전시회_관람   0.061243  ...  0.036763  0.019322  0.046132  0.061018   
         디자인_전시회_관람  0.087331  ...  0.040732  0.018774  0.058015  0.078377   
         박물관_관람      0.063549  ...  0.039723  0.023093  0.063719  0.074317   
...                       ...  ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.070697  ...  0.094238  0.033590  0.338769  0.265950   
         이성교제        0.034983  ...  0.028755  0.007881  0.030439  0.095203   
         미팅          0.054314  ...  0.100068  0.050936  0.210962  0.173244   
         소개팅         0.045617  ...  0.056760  0.025659  0.346033  0.116611   
         동호회_모임      0.058234  ...  0.055638  0.038535  0.164293  0.194622   

                                                                       \
                         사교파티     친구_만남      이성교제        미팅       소개팅   
문화예술관람활동 미술_전시회_관람   0.040349  0.084373  0.031545  0.108232  0.052813   
         사진_전시회_관람   0.047474  0.119339  0.034713  0.134054  0.078619   
         건축_전시회_관람   0.043945  0.074717  0.024502  0.205247  0.062098   
         디자인_전시회_관람  0.053924  0.109248  0.033300  0.166119  0.086900   
         박물관_관람      0.041932  0.129643  0.032987  0.120814  0.066583   
...                       ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.097003  1.000000  0.104948  0.264878  0.329443   
         이성교제        0.042263  0.104948  1.000000  0.075669  0.122431   
         미팅          0.262378  0.264878  0.075669  1.000000  0.347371   
         소개팅         0.202387  0.329443  0.122431  0.347371  1.000000   
         동호회_모임      0.272270  0.248170  0.041104  0.289931  0.283431   

                               
                       동호회_모임  
문화예술관람활동 미술_전시회_관람   0.165624  
         사진_전시회_관람   0.171160  
         건축_전시회

In [156]:
for index,row in enumerate(leisure_dict_search):
    for obj in row:
        correlation[big_cat[index],obj].loc[big_cat[index],obj]=0 #각 대각값을 0으로 변경
correlation

문화예술관람활동                                           \
                    미술_전시회_관람 사진_전시회_관람 건축_전시회_관람 디자인_전시회_관람    박물관_관람   
문화예술관람활동 미술_전시회_관람   0.000000  0.901811  0.608886   0.781737  0.422173   
         사진_전시회_관람   0.901811  0.000000  0.627134   0.831878  0.481548   
         건축_전시회_관람   0.608886  0.627134  0.000000   0.682047  0.388383   
         디자인_전시회_관람  0.781737  0.831878  0.682047   0.000000  0.460597   
         박물관_관람      0.422173  0.481548  0.388383   0.460597  0.000000   
...                       ...       ...       ...        ...       ...   
사회/기타활동  친구_만남       0.084373  0.119339  0.074717   0.109248  0.129643   
         이성교제        0.031545  0.034713  0.024502   0.033300  0.032987   
         미팅          0.108232  0.134054  0.205247   0.166119  0.120814   
         소개팅         0.052813  0.078619  0.062098   0.086900  0.066583   
         동호회_모임      0.165624  0.171160  0.133195   0.162514  0.133329   

                                                                       \
                    클래식_음악_연주회_관람 오페라_음악_연주회_관람 국악_전통공연_관람 민속놀이_공연_관람   
문화예술관람활동 미술_전시회_관람       0.153628      0.149917   0.121489   0.220757   
         사진_전시회_관람       0.145338      0.139008   0.113648   0.241190   
         건축_전시회_관람       0.112477      0.108082   0.099555   0.219725   
         디자인_전시회_관람      0.139484      0.133906   0.106327   0.227851   
         박물관_관람          0.105126      0.104950   0.174615   0.354507   
...                           ...           ...        ...        ...   
사회/기타활동  친구_만남           0.051113      0.043145   0.043504   0.073876   
         이성교제            0.028968      0.029836   0.015472   0.022057   
         미팅              0.056009      0.042689   0.054082   0.093369   
         소개팅             0.027096      0.021939   0.020806   0.045822   
         동호회_모임          0.069557      0.069696   0.056026   0.107897   

                               ...   사회/기타활동                                \
                      연극공연_관람  ...      통화하기     문자보내기       계모임       동창회   
문화예술관람활동 미술_전시회_관람   0.088846  ...  0.029996  0.015479  0.039317  0.059336   
         사진_전시회_관람   0.099477  ...  0.037214  0.018288  0.056720  0.070297   
         건축_전시회_관람   0.061243  ...  0.036763  0.019322  0.046132  0.061018   
         디자인_전시회_관람  0.087331  ...  0.040732  0.018774  0.058015  0.078377   
         박물관_관람      0.063549  ...  0.039723  0.023093  0.063719  0.074317   
...                       ...  ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.070697  ...  0.094238  0.033590  0.338769  0.265950   
         이성교제        0.034983  ...  0.028755  0.007881  0.030439  0.095203   
         미팅          0.054314  ...  0.100068  0.050936  0.210962  0.173244   
         소개팅         0.045617  ...  0.056760  0.025659  0.346033  0.116611   
         동호회_모임      0.058234  ...  0.055638  0.038535  0.164293  0.194622   

                                                                       \
                         사교파티     친구_만남      이성교제        미팅       소개팅   
문화예술관람활동 미술_전시회_관람   0.040349  0.084373  0.031545  0.108232  0.052813   
         사진_전시회_관람   0.047474  0.119339  0.034713  0.134054  0.078619   
         건축_전시회_관람   0.043945  0.074717  0.024502  0.205247  0.062098   
         디자인_전시회_관람  0.053924  0.109248  0.033300  0.166119  0.086900   
         박물관_관람      0.041932  0.129643  0.032987  0.120814  0.066583   
...                       ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.097003  0.000000  0.104948  0.264878  0.329443   
         이성교제        0.042263  0.104948  0.000000  0.075669  0.122431   
         미팅          0.262378  0.264878  0.075669  0.000000  0.347371   
         소개팅         0.202387  0.329443  0.122431  0.347371  0.000000   
         동호회_모임      0.272270  0.248170  0.041104  0.289931  0.283431   

                               
                       동호회_모임  
문화예술관람활동 미술_전시회_관람   0.165624  
         사진_전시회_관람   0.171160  
         건축_전시회

In [44]:
correlation['문화예술관광활동']*1.2 #곱하고
correlation.loc['문화예술관광활동'] #곱하고
correlation['문화예술관광활동'].loc['문화예술관광활동'] #겹치는 부분 나누어주기

,미술_전시회_관람,사진_전시회_관람,건축_전시회_관람,디자인_전시회_관람,박물관_관람,클래식_음악_연주회_관람,오페라_음악_연주회_관람,국악_전통공연_관람,민속놀이_공연_관람,연극공연_관람,뮤지컬_관람,무용_공연_관람,영화_관람,쇼_공연_관람,콘서트_공연_관람,마술_쇼_공연_관람
미술_전시회_관람,1.000000,0.901811,0.608886,0.781737,0.422173,0.153628,0.149917,0.121489,0.220757,0.088846,0.126218,0.318603,0.111610,0.098201,0.156513,0.078163
사진_전시회_관람,0.901811,1.000000,0.627134,0.831878,0.481548,0.145338,0.139008,0.113648,0.241190,0.099477,0.162824,0.299755,0.146500,0.134418,0.194186,0.095003
건축_전시회_관람,0.608886,0.627134,1.000000,0.682047,0.388383,0.112477,0.108082,0.099555,0.219725,0.061243,0.090349,0.225951,0.090003,0.096016,0.138370,0.071697
디자인_전시회_관람,0.781737,0.831878,0.682047,1.000000,0.460597,0.139484,0.133906,0.106327,0.227851,0.087331,0.141181,0.281988,0.127487,0.130726,0.173944,0.096574
박물관_관람,0.422173,0.481548,0.388383,0.460597,1.000000,0.105126,0.104950,0.174615,0.354507,0.063549,0.134308,0.196822,0.113233,0.157589,0.142980,0.125712
클래식_음악_연주회_관람,0.153628,0.145338,0.112477,0.139484,0.105126,1.000000,0.875544,0.431468,0.126836,0.202161,0.307845,0.304546,0.088896,0.113065,0.411732,0.070936
오페라_음악_연주회_관람,0.149917,0.139008,0.108082,0.133906,0.104950,0.875544,1.000000,0.403500,0.123860,0.214385,0.269959,0.312634,0.085364,0.109595,0.383264,0.064065
국악_전통공연_관람,0.121489,0.113648,0.099555,0.106327,0.174615,0.431468,0.403500,1.000000,0.303052,0.385552,0.534679,0.331848,0.077693,0.151003,0.311203,0.092648
민속놀이_공연_관람,0.220757,0.241190,0.219725,0.227851,0.354507,0.126836,0.123860,0.303052,1.000000,0.060653,0.116290,0.277741,0.082337,0.173609,0.206307,0.142111
연극공연_관람,0.088846,0.099477,0.061243,0.087331,0.063549,0.202161,0.214385,0.385552,0.060653,1.000000,0.506911,0.320325,0.142803,0.203688,0.258423,0.090251


- 가중치 산정 방식
  - 사용자에게 최소 1개, 최대 3개의 취미 + 성별 + 연령대 + 주말/휴일 여부를 물어본다.
  - 각 취미별로 상위 5개 상관도에 1.5, 1.4, 1.3, 1.2, 1.1의 가중치를 부여
  - 해당되는 주말/휴일 그래프에 성별별 활동에 다음을 부여
    - 1-2등 활동에 1.2 가중치, 3-4등 활동에 1.1 가중치, 그 외의 활동 중 퍼센트가 50미만인 활동은 0.9를 부여
  - 해당되는 주말/휴일 그래프에 연령별 활동에 다음을 부여
    - 1-2등 활동에 1.2 가중치, 3-4등 활동에 1.1 가중치, 그 외의 활동 중 퍼센트가 50미만인 활동은 0.9를 부여
  - 만들어진 상관도 표를 통해 온라인 별, 오프라인 별, 온/오프라인 별, 여가유형별 상위 활동을 출력한다.

In [142]:
correlation['휴식활동']

산책        걷기        목욕       사우나       찜질방  \
문화예술관람활동 미술_전시회_관람   0.102123  0.060003  0.023537  0.054686  0.043009   
         사진_전시회_관람   0.132379  0.081812  0.034422  0.079803  0.065208   
         건축_전시회_관람   0.101629  0.058546  0.035574  0.100004  0.066591   
         디자인_전시회_관람  0.126629  0.075981  0.043525  0.095841  0.067018   
         박물관_관람      0.197303  0.126391  0.041876  0.101839  0.083274   
...                       ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.216029  0.199381  0.098694  0.128882  0.104796   
         이성교제        0.028778  0.027978  0.015171  0.019804  0.017570   
         미팅          0.114076  0.097762  0.043898  0.157467  0.085692   
         소개팅         0.104184  0.076680  0.026472  0.144282  0.076749   
         동호회_모임      0.138284  0.127960  0.033912  0.117961  0.074139   

                           낮잠     TV_시청   모바일_컨텐츠    동영상_시청    라디오_청취  \
문화예술관람활동 미술_전시회_관람   0.019174  0.063956  0.052685  0.073456  0.041181   
         사진_전시회_관람   0.030797  0.073726  0.072602  0.088402  0.058869   
         건축_전시회_관람   0.028359  0.076654  0.110676  0.091960  0.072000   
         디자인_전시회_관람  0.039237  0.086211  0.094088  0.098767  0.075741   
         박물관_관람      0.029073  0.095074  0.074348  0.102965  0.051713   
...                       ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.087483  0.092704  0.043300  0.073881  0.051757   
         이성교제        0.010872  0.051827  0.023789  0.054945  0.019981   
         미팅          0.040139  0.151201  0.190109  0.207156  0.109107   
         소개팅         0.025183  0.047336  0.052874  0.064276  0.037356   
         동호회_모임      0.029523  0.103655  0.105681  0.127789  0.061738   

                      팟캐스트_청취     음악_감상     신문_보기     잡지_보기  아무것도_안_하기  
문화예술관람활동 미술_전시회_관람   0.091755  0.084557  0.039308  0.104723   0.086787  
         사진_전시회_관람   0.109992  0.104163  0.041889  0.115567   0.107585  
         건축_전시회_관람   0.085911  0.076571  0.081491  0.108558   0.058932  
         디자인_전시회_관람  0.104870  0.105907  0.044477  0.125633   0.088549  
         박물관_관람      0.091817  0.070701  0.070322  0.112319   0.103970  
...                       ...       ...       ...       ...        ...  
사회/기타활동  친구_만남       0.115822  0.080562  0.046230  0.109007   0.360481  
         이성교제        0.059259  0.028643  0.033152  0.043209   0.073160  
         미팅          0.175799  0.114440  0.092533  0.118020   0.163396  
         소개팅         0.059358  0.057668  0.026841  0.057744   0.107019  
         동호회_모임      0.096868  0.101193  0.049366  0.098460   0.150637  

[211 rows x 15 columns]

In [542]:
#사용자 전용 복제품 생성
rep_correlation=correlation.copy()
rep_correlation

문화예술관람활동                                           \
                    미술_전시회_관람 사진_전시회_관람 건축_전시회_관람 디자인_전시회_관람    박물관_관람   
문화예술관람활동 미술_전시회_관람   0.000000  0.901811  0.608886   0.781737  0.422173   
         사진_전시회_관람   0.901811  0.000000  0.627134   0.831878  0.481548   
         건축_전시회_관람   0.608886  0.627134  0.000000   0.682047  0.388383   
         디자인_전시회_관람  0.781737  0.831878  0.682047   0.000000  0.460597   
         박물관_관람      0.422173  0.481548  0.388383   0.460597  0.000000   
...                       ...       ...       ...        ...       ...   
사회/기타활동  친구_만남       0.084373  0.119339  0.074717   0.109248  0.129643   
         이성교제        0.031545  0.034713  0.024502   0.033300  0.032987   
         미팅          0.108232  0.134054  0.205247   0.166119  0.120814   
         소개팅         0.052813  0.078619  0.062098   0.086900  0.066583   
         동호회_모임      0.165624  0.171160  0.133195   0.162514  0.133329   

                                                                       \
                    클래식_음악_연주회_관람 오페라_음악_연주회_관람 국악_전통공연_관람 민속놀이_공연_관람   
문화예술관람활동 미술_전시회_관람       0.153628      0.149917   0.121489   0.220757   
         사진_전시회_관람       0.145338      0.139008   0.113648   0.241190   
         건축_전시회_관람       0.112477      0.108082   0.099555   0.219725   
         디자인_전시회_관람      0.139484      0.133906   0.106327   0.227851   
         박물관_관람          0.105126      0.104950   0.174615   0.354507   
...                           ...           ...        ...        ...   
사회/기타활동  친구_만남           0.051113      0.043145   0.043504   0.073876   
         이성교제            0.028968      0.029836   0.015472   0.022057   
         미팅              0.056009      0.042689   0.054082   0.093369   
         소개팅             0.027096      0.021939   0.020806   0.045822   
         동호회_모임          0.069557      0.069696   0.056026   0.107897   

                               ...   사회/기타활동                                \
                      연극공연_관람  ...      통화하기     문자보내기       계모임       동창회   
문화예술관람활동 미술_전시회_관람   0.088846  ...  0.029996  0.015479  0.039317  0.059336   
         사진_전시회_관람   0.099477  ...  0.037214  0.018288  0.056720  0.070297   
         건축_전시회_관람   0.061243  ...  0.036763  0.019322  0.046132  0.061018   
         디자인_전시회_관람  0.087331  ...  0.040732  0.018774  0.058015  0.078377   
         박물관_관람      0.063549  ...  0.039723  0.023093  0.063719  0.074317   
...                       ...  ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.070697  ...  0.094238  0.033590  0.338769  0.265950   
         이성교제        0.034983  ...  0.028755  0.007881  0.030439  0.095203   
         미팅          0.054314  ...  0.100068  0.050936  0.210962  0.173244   
         소개팅         0.045617  ...  0.056760  0.025659  0.346033  0.116611   
         동호회_모임      0.058234  ...  0.055638  0.038535  0.164293  0.194622   

                                                                       \
                         사교파티     친구_만남      이성교제        미팅       소개팅   
문화예술관람활동 미술_전시회_관람   0.040349  0.084373  0.031545  0.108232  0.052813   
         사진_전시회_관람   0.047474  0.119339  0.034713  0.134054  0.078619   
         건축_전시회_관람   0.043945  0.074717  0.024502  0.205247  0.062098   
         디자인_전시회_관람  0.053924  0.109248  0.033300  0.166119  0.086900   
         박물관_관람      0.041932  0.129643  0.032987  0.120814  0.066583   
...                       ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.097003  0.000000  0.104948  0.264878  0.329443   
         이성교제        0.042263  0.104948  0.000000  0.075669  0.122431   
         미팅          0.262378  0.264878  0.075669  0.000000  0.347371   
         소개팅         0.202387  0.329443  0.122431  0.347371  0.000000   
         동호회_모임      0.272270  0.248170  0.041104  0.289931  0.283431   

                               
                       동호회_모임  
문화예술관람활동 미술_전시회_관람   0.165624  
         사진_전시회_관람   0.171160  
         건축_전시회

In [264]:
rep_correlation['휴식활동']=rep_correlation['휴식활동'].apply(lambda x:x*1.2)
rep_correlation['휴식활동']

산책        걷기        목욕       사우나       찜질방  \
문화예술관람활동 미술_전시회_관람   0.304937  0.179167  0.070280  0.163292  0.128423   
         사진_전시회_관람   0.395280  0.244289  0.102783  0.238290  0.194709   
         건축_전시회_관람   0.303461  0.174817  0.106223  0.298612  0.198840   
         디자인_전시회_관람  0.378113  0.226877  0.129965  0.286179  0.200115   
         박물관_관람      0.589145  0.377401  0.125041  0.304089  0.248655   
...                       ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.645060  0.595349  0.294699  0.384838  0.312918   
         이성교제        0.085931  0.083541  0.045299  0.059133  0.052463   
         미팅          0.340629  0.291917  0.131078  0.470194  0.255874   
         소개팅         0.311091  0.228964  0.079045  0.430823  0.229171   
         동호회_모임      0.412913  0.382087  0.101262  0.352229  0.221378   

                           낮잠     TV_시청   모바일_컨텐츠    동영상_시청    라디오_청취  \
문화예술관람활동 미술_전시회_관람   0.057253  0.190972  0.157315  0.219338  0.122964   
         사진_전시회_관람   0.091958  0.220146  0.216789  0.263966  0.175781   
         건축_전시회_관람   0.084679  0.228887  0.330475  0.274592  0.214991   
         디자인_전시회_관람  0.117160  0.257425  0.280945  0.294917  0.226162   
         박물관_관람      0.086810  0.283888  0.222002  0.307452  0.154415   
...                       ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.261224  0.276812  0.129293  0.220608  0.154545   
         이성교제        0.032463  0.154756  0.071032  0.164064  0.059664   
         미팅          0.119856  0.451484  0.567663  0.618564  0.325791   
         소개팅         0.075197  0.141343  0.157881  0.191927  0.111543   
         동호회_모임      0.088155  0.309511  0.315562  0.381577  0.184348   

                      팟캐스트_청취     음악_감상     신문_보기     잡지_보기  아무것도_안_하기  
문화예술관람활동 미술_전시회_관람   0.273979  0.252485  0.117374  0.312702   0.259145  
         사진_전시회_관람   0.328436  0.311028  0.125079  0.345082   0.321247  
         건축_전시회_관람   0.256527  0.228640  0.243331  0.324152   0.175969  
         디자인_전시회_관람  0.313141  0.316235  0.132807  0.375137   0.264407  
         박물관_관람      0.274164  0.211113  0.209981  0.335383   0.310453  
...                       ...       ...       ...       ...        ...  
사회/기타활동  친구_만남       0.345841  0.240557  0.138041  0.325493   1.076392  
         이성교제        0.176947  0.085526  0.098992  0.129021   0.218455  
         미팅          0.524933  0.341717  0.276303  0.352405   0.487898  
         소개팅         0.177243  0.172197  0.080147  0.172422   0.319556  
         동호회_모임      0.289247  0.302161  0.147406  0.293999   0.449800  

[211 rows x 15 columns]

In [576]:
def weight_cal(correlation, list_, df):
    for key,value in enumerate(list_):
        weight=1
        if key<2:
            weight=1.2
        elif key<4:
            weight=1.1
        elif df[value]<50:
            weight=0.9
        correlation[value]=correlation[value].apply(lambda x:x*weight)

def week_type(week):#1이 주말, 2가 평일. 데이터프레임 반환
    if week==1 or week=='주말':
        return weekend
    elif week==2 or week=='평일':
        return weekday
    return False
        
def sex(correlation, sex, week):
    list_=[]
    if sex==1 or sex=='남성':#남성을 입력받음
        sexdf=week.loc['남성'].sort_values(ascending=False)
    elif sex==2 or sex=='여성':#여성을 입력받음
        sexdf=week.loc['여성'].sort_values(ascending=False)
    elif sex==-1:#입력을 원하지 않음
        return 
    else: #잘못된 값
        return False
    list_=list(sexdf.index)
    weight_cal(correlation, list_, sexdf)
        
def age(correlation, age, week):
    list_=[]
    if age==1 or age=='10대':
        agedf=week.loc['15-19세'].sort_values(ascending=False)
    elif age==2 or age=='20대':
        agedf=week.loc['20대'].sort_values(ascending=False)
    elif age==3 or age=='30대':
        agedf=week.loc['30대'].sort_values(ascending=False)
    elif age==4 or age=='40대':
        agedf=week.loc['40대'].sort_values(ascending=False)
    elif age==5 or age=='50대':
        agedf=week.loc['50대'].sort_values(ascending=False)
    elif age==6 or age=='60대':
        agedf=week.loc['60대'].sort_values(ascending=False)
    elif age==7 or age=='70대이상':
        agedf=week.loc['70대이상'].sort_values(ascending=False)
    elif age==-1:#입력을 원하지 않음
        return
    else: #잘못된 값
        return False
    list_=list(agedf.index)
    weight_cal(correlation, list_, agedf)

def prefer_weight(correlation):
    print(type(correlation))
    for i,obj in enumerate(correlation.columns[:-1]):
        correlation=correlation.sort_values(by=obj, ascending=False)
        correlation.iloc[0,i]=correlation.iloc[0,i]*1.5
        correlation.iloc[1,i]=correlation.iloc[1,i]*1.4
        correlation.iloc[2,i]=correlation.iloc[2,i]*1.3
        correlation.iloc[3,i]=correlation.iloc[3,i]*1.2
        correlation.iloc[4,i]=correlation.iloc[4,i]*1.1
    print(type(correlation))
    return correlation

In [543]:
sex(rep_correlation, 1, weekday)
#rep_correlation['휴식활동'].loc['휴식활동']
rep_correlation

문화예술관람활동                                           \
                    미술_전시회_관람 사진_전시회_관람 건축_전시회_관람 디자인_전시회_관람    박물관_관람   
문화예술관람활동 미술_전시회_관람   0.000000  0.811630  0.547997   0.703564  0.379956   
         사진_전시회_관람   0.811630  0.000000  0.564420   0.748690  0.433393   
         건축_전시회_관람   0.547997  0.564420  0.000000   0.613843  0.349545   
         디자인_전시회_관람  0.703564  0.748690  0.613843   0.000000  0.414537   
         박물관_관람      0.379956  0.433393  0.349545   0.414537  0.000000   
...                       ...       ...       ...        ...       ...   
사회/기타활동  친구_만남       0.075935  0.107406  0.067245   0.098324  0.116678   
         이성교제        0.028391  0.031241  0.022052   0.029970  0.029688   
         미팅          0.097408  0.120648  0.184723   0.149507  0.108732   
         소개팅         0.047532  0.070757  0.055889   0.078210  0.059925   
         동호회_모임      0.149062  0.154044  0.119875   0.146263  0.119997   

                                                                       \
                    클래식_음악_연주회_관람 오페라_음악_연주회_관람 국악_전통공연_관람 민속놀이_공연_관람   
문화예술관람활동 미술_전시회_관람       0.138265      0.134926   0.109340   0.198681   
         사진_전시회_관람       0.130804      0.125108   0.102283   0.217071   
         건축_전시회_관람       0.101229      0.097274   0.089600   0.197752   
         디자인_전시회_관람      0.125536      0.120515   0.095694   0.205066   
         박물관_관람          0.094613      0.094455   0.157154   0.319056   
...                           ...           ...        ...        ...   
사회/기타활동  친구_만남           0.046002      0.038830   0.039154   0.066488   
         이성교제            0.026071      0.026852   0.013925   0.019851   
         미팅              0.050408      0.038420   0.048674   0.084033   
         소개팅             0.024387      0.019745   0.018726   0.041240   
         동호회_모임          0.062601      0.062726   0.050423   0.097107   

                               ...   사회/기타활동                                \
                      연극공연_관람  ...      통화하기     문자보내기       계모임       동창회   
문화예술관람활동 미술_전시회_관람   0.079962  ...  0.032995  0.017027  0.043248  0.065270   
         사진_전시회_관람   0.089529  ...  0.040935  0.020116  0.062392  0.077326   
         건축_전시회_관람   0.055119  ...  0.040439  0.021254  0.050746  0.067120   
         디자인_전시회_관람  0.078598  ...  0.044805  0.020652  0.063817  0.086215   
         박물관_관람      0.057194  ...  0.043696  0.025402  0.070091  0.081749   
...                       ...  ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.063628  ...  0.103662  0.036949  0.372646  0.292545   
         이성교제        0.031485  ...  0.031630  0.008669  0.033483  0.104724   
         미팅          0.048883  ...  0.110075  0.056030  0.232058  0.190568   
         소개팅         0.041055  ...  0.062437  0.028225  0.380636  0.128272   
         동호회_모임      0.052411  ...  0.061202  0.042388  0.180722  0.214084   

                                                                       \
                         사교파티     친구_만남      이성교제        미팅       소개팅   
문화예술관람활동 미술_전시회_관람   0.044384  0.092810  0.034700  0.119055  0.058095   
         사진_전시회_관람   0.052222  0.131273  0.038184  0.147459  0.086481   
         건축_전시회_관람   0.048340  0.082188  0.026952  0.225772  0.068308   
         디자인_전시회_관람  0.059316  0.120173  0.036630  0.182731  0.095590   
         박물관_관람      0.046125  0.142607  0.036286  0.132895  0.073241   
...                       ...       ...       ...       ...       ...   
사회/기타활동  친구_만남       0.106703  0.000000  0.115443  0.291366  0.362387   
         이성교제        0.046489  0.115443  0.000000  0.083236  0.134675   
         미팅          0.288616  0.291366  0.083236  0.000000  0.382108   
         소개팅         0.222626  0.362387  0.134675  0.382108  0.000000   
         동호회_모임      0.299497  0.272987  0.045215  0.318925  0.311774   

                               
                       동호회_모임  
문화예술관람활동 미술_전시회_관람   0.182187  
         사진_전시회_관람   0.188276  
         건축_전시회

In [573]:
data1=rep_correlation['휴식활동','아무것도_안_하기']
data2=rep_correlation['문화예술관람활동','미술_전시회_관람']
data3=rep_correlation['사회/기타활동','통화하기']

In [519]:
fread=open("./Leisures/Description_Data_New/onoffdatas.pkl", 'rb')
templist=pickle.load(fread)
fread.close()
onoff=Series(data=templist, index=columns)

In [574]:
test=pd.DataFrame([data1,data2,data3]).T
test['OnOff']=onoff
test

(휴식활동, 아무것도_안_하기)  (문화예술관람활동, 미술_전시회_관람)  \
문화예술관람활동 미술_전시회_관람            0.104145               0.000000   
         사진_전시회_관람            0.129102               0.811630   
         건축_전시회_관람            0.070718               0.547997   
         디자인_전시회_관람           0.106259               0.703564   
         박물관_관람               0.124764               0.379956   
...                                ...                    ...   
사회/기타활동  친구_만남                0.432578               0.075935   
         이성교제                 0.087792               0.028391   
         미팅                   0.196075               0.097408   
         소개팅                  0.128423               0.047532   
         동호회_모임               0.180765               0.149062   

                     (사회/기타활동, 통화하기)  OnOff  
문화예술관람활동 미술_전시회_관람          0.032995      1  
         사진_전시회_관람          0.040935      1  
         건축_전시회_관람          0.040439      1  
         디자인_전시회_관람         0.044805      1  
         박물관_관람             0.043696      1  
...                              ...    ...  
사회/기타활동  친구_만남              0.103662      2  
         이성교제               0.031630      1  
         미팅                 0.110075      2  
         소개팅                0.062437      1  
         동호회_모임             0.061202      1  

[211 rows x 4 columns]

In [537]:
print(type(test.columns[0]))

<class 'tuple'>


In [566]:
templist=test.columns
test.sort_values(by=templist[2], ascending=False)

(휴식활동, 아무것도_안_하기)  (문화예술관람활동, 미술_전시회_관람)  (사회/기타활동, 통화하기)  \
휴식활동    음악_감상            0.102419               0.076101         0.271325   
        동영상_시청           0.127996               0.066110         0.177756   
        라디오_청취           0.066583               0.037062         0.175177   
사회/기타활동 문자보내기            0.037919               0.013931         0.158098   
휴식활동    TV_시청            0.133431               0.057561         0.140404   
...                           ...                    ...              ...   
취미/오락활동 비즈공예             0.021148               0.026069         0.012581   
        네일아트             0.026475               0.071522         0.011729   
사회/기타활동 종교활동             0.024047               0.015330         0.011681   
취미/오락활동 음주               0.051075               0.007960         0.011109   
사회/기타활동 통화하기             0.090211               0.026996         0.000000   

                OnOff  
휴식활동    음악_감상       2  
        동영상_시청      0  
        라디오_청취      2  
사회/기타활동 문자보내기       0  
휴식활동    TV_시청       0  
...               ...  
취미/오락활동 비즈공예        0  
        네일아트        2  
사회/기타활동 종교활동        1  
취미/오락활동 음주          1  
사회/기타활동 통화하기        0  

[211 rows x 4 columns]

In [579]:
weightresult=prefer_weight(test)
weightresult

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


(휴식활동, 아무것도_안_하기)  (문화예술관람활동, 미술_전시회_관람)  (사회/기타활동, 통화하기)  \
휴식활동    음악_감상            0.102419               0.076101         0.406988   
        동영상_시청           0.127996               0.066110         0.248859   
        라디오_청취           0.066583               0.037062         0.227730   
사회/기타활동 문자보내기            0.037919               0.013931         0.189718   
휴식활동    TV_시청            0.133431               0.057561         0.154444   
...                           ...                    ...              ...   
취미/오락활동 비즈공예             0.021148               0.026069         0.012581   
        네일아트             0.026475               0.071522         0.011729   
사회/기타활동 종교활동             0.024047               0.015330         0.011681   
취미/오락활동 음주               0.051075               0.007960         0.011109   
사회/기타활동 통화하기             0.090211               0.026996         0.000000   

                OnOff  
휴식활동    음악_감상       2  
        동영상_시청      0  
        라디오_청취      2  
사회/기타활동 문자보내기       0  
휴식활동    TV_시청       0  
...               ...  
취미/오락활동 비즈공예        0  
        네일아트        2  
사회/기타활동 종교활동        1  
취미/오락활동 음주          1  
사회/기타활동 통화하기        0  

[211 rows x 4 columns]

- 각 값의 최대값으로 압축한다면?

In [581]:
weightresult['max']=weightresult.drop('OnOff',axis=1).max(axis=1)
weightresult

(휴식활동, 아무것도_안_하기)  (문화예술관람활동, 미술_전시회_관람)  (사회/기타활동, 통화하기)  \
휴식활동    음악_감상            0.102419               0.076101         0.406988   
        동영상_시청           0.127996               0.066110         0.248859   
        라디오_청취           0.066583               0.037062         0.227730   
사회/기타활동 문자보내기            0.037919               0.013931         0.189718   
휴식활동    TV_시청            0.133431               0.057561         0.154444   
...                           ...                    ...              ...   
취미/오락활동 비즈공예             0.021148               0.026069         0.012581   
        네일아트             0.026475               0.071522         0.011729   
사회/기타활동 종교활동             0.024047               0.015330         0.011681   
취미/오락활동 음주               0.051075               0.007960         0.011109   
사회/기타활동 통화하기             0.090211               0.026996         0.000000   

                OnOff       max  
휴식활동    음악_감상       2  0.406988  
        동영상_시청      0  0.248859  
        라디오_청취      2  0.227730  
사회/기타활동 문자보내기       0  0.189718  
휴식활동    TV_시청       0  0.154444  
...               ...       ...  
취미/오락활동 비즈공예        0  0.026069  
        네일아트        2  0.071522  
사회/기타활동 종교활동        1  0.024047  
취미/오락활동 음주          1  0.051075  
사회/기타활동 통화하기        0  0.090211  

[211 rows x 5 columns]

In [582]:
result1=list(weightresult[weightresult['OnOff']!=1]['max'].sort_values(ascending=False)[:5].items()) #오프라인이 아니면서 상관도가 제일 높은 상위 5개
print(result1)

[(('사회/기타활동', '친구_만남'), 0.6488665910687132), (('사회/기타활동', '잡담'), 0.4674265108333899), (('휴식활동', '음악_감상'), 0.4069880677580488), (('문화예술참여활동', '도예_활동_참여'), 0.3532830484414088), (('문화예술참여활동', '만화_활동_참여'), 0.34125214093101036)]


In [583]:
result2=list(weightresult[weightresult['OnOff']!=0]['max'].sort_values(ascending=False)[:5].items()) #온라인이 아니면서 상관도가 제일 높은 상위 5개
print(result2)

[(('문화예술관람활동', '사진_전시회_관람'), 1.217445316918043), (('문화예술관람활동', '디자인_전시회_관람'), 0.984989030119654), (('관광활동', '풍경_관람'), 0.8292506519853297), (('문화예술참여활동', '조각_활동_참여'), 0.7114372895216977), (('사회/기타활동', '친구_만남'), 0.6488665910687132)]


In [584]:
result3=list(weightresult['max'].sort_values(ascending=False)[:5].items()) #온/오프라인 유무 없이 상관도가 제일 높은 상위 5개
print(result3)

[(('문화예술관람활동', '사진_전시회_관람'), 1.217445316918043), (('문화예술관람활동', '디자인_전시회_관람'), 0.984989030119654), (('관광활동', '풍경_관람'), 0.8292506519853297), (('문화예술참여활동', '조각_활동_참여'), 0.7114372895216977), (('사회/기타활동', '친구_만남'), 0.6488665910687132)]


In [585]:
result4=[]
for i in big_cat:
    result4.append(list(weightresult.loc[i]['max'].sort_values(ascending=False)[:1].items())) #카테고리별 상관도가 제일 높은 상위 1개

In [586]:
print(result4)

[[('사진_전시회_관람', 1.217445316918043)], [('조각_활동_참여', 0.7114372895216977)], [('축구_경기_티비_관람', 0.23037545071879179)], [('포켓볼_참여', 0.2572236872299161)], [('풍경_관람', 0.8292506519853297)], [('독서', 0.2166646714764574)], [('음악_감상', 0.4069880677580488)], [('친구_만남', 0.6488665910687132)]]


# 이제 실전이다. 사용자 입장에서 보자

In [ ]:
flag=False
week_list=[1,2]
week_str=["주말","휴일"]
while flag is False:
    week=input("어느 날에 취미를 가지실건가요? : ")
    if (week in week_list) or (week in week_str):
        weekdf=week_type(week)
        flag=True
    else:
        print("잘못된 입력입니다.")

In [ ]:
flag=False
sex_list=[1,2]
sex_str=["남성","여성"]

while flag is False:
    sex=input("성별을 입력해주세요 : ")
    if (sex in sex_list) or (sex in sex_str) or sex==-1:
        flag=True
        
    else:
        print("잘못된 입력입니다.")

In [589]:
flag=False
age_list=[1,2,3,4,5,6,7]
age_str=["10대","20대","30대","40대","50대","60대","70대이상"]
while flag is False:
    age=input("나이를 입력해주세요 : ")
    if (age in age_list) or (age in age_str) or age==-1:
        flag=True
    else:
        print("잘못된 입력입니다.")

나이를 입력해주세요 : 8
잘못된 입력입니다.
나이를 입력해주세요 : 8
잘못된 입력입니다.
나이를 입력해주세요 : 9
잘못된 입력입니다.
나이를 입력해주세요 : 10
잘못된 입력입니다.
나이를 입력해주세요 : 10대
